In [1]:
import numpy as np
import netCDF4

from scipy import (special, signal, ndimage)
from scipy.ndimage import gaussian_filter

from ipywidgets import (interact)

import bokeh
from bokeh.plotting import (figure, show, output_notebook)
from bokeh.models import (LinearColorMapper, Ticker, ColorBar)
from bokeh.io import (push_notebook)

import os

output_notebook()

path = "/mnt/Knolselderij/bulk/Abrupt/"
fn1 = "tas_Amon_MPI-ESM-LR_rcp85_r1i1p1_200601-210012.nc"
fn2 = "tas_Amon_MPI-ESM-LR_rcp85_r1i1p1_210101-230012.nc"

Loading BokehJS ...

In [2]:
def get_data(filename):
    return netCDF4.Dataset(filename, 'r', format='NETCDF4')

def plot_image(data):
    ratio = data.shape[0] / data.shape[1]
    p = figure(x_range=(0, data.shape[1]), y_range=(0, data.shape[0]), plot_width=int(378/ratio), plot_height=378)
    p.image(image=[data], x=0, y=0, dw=data.shape[1], dh=data.shape[0], palette=bokeh.palettes.viridis(255))
    show(p)

data = np.concatenate([
    get_data(os.path.join(path, fn1)).variables['tas'],
    get_data(os.path.join(path, fn2)).variables['tas']],
    axis=0)

## Compute the variance of the data
This to assess the magnitude of something that should be considered _extreme_.

In [3]:
control_files = [
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_185001-203512.nc",
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_203601-213012.nc",
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_213101-223012.nc",
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_223101-233012.nc",
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_233101-249912.nc",
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_250001-269912.nc",
    "tas_Amon_MPI-ESM-LR_piControl_r1i1p1_270001-284912.nc"]

In [4]:
def compute_std(files, variable):
    sum_lin = 0
    sum_sqr = 0
    n = 0

    for f in files:
        d = get_data(os.path.join(path, f)).variables[variable][:]
        sum_lin += d.sum(axis=0)
        sum_sqr += (d**2).sum(axis=0)
        n += d.shape[0]
        del d
    
    return np.sqrt(sum_sqr/n - (sum_lin/n)**2)

In [5]:
std = compute_std(control_files, 'tas')
plot_image(std)

## Canny edges

In [6]:
from copy import copy

def is_linear(a, eps=1e-3):
    x = np.diff(a[1:-1]).std() / np.diff(a[1:-1]).mean()
    return x < eps


R_earth = 6.371e3             # km
day = 0.0027378507871321013   # years


class Box:
    def __init__(self, nc):
        self.lat = nc.variables['lat']
        self.lon = nc.variables['lon']
        self.time = nc.variables['time']
    
    def __getitem__(self, s):
        new_box = copy(self)
        
        if not isinstance(s, tuple):
            s = (s,)
        
        for q, t in zip(s, ['time', 'lat', 'lon']):
            setattr(new_box, t, getattr(self, t).__getitem__(s))
        
        return new_box
            
    @property
    def rectangular(self):
        return is_linear(self.lat) and is_linear(self.lon)
    
    @property
    def resolution(self):
        res_lat = np.diff(self.lat[1:-1]).mean() * (np.pi / 180) * R_earth
        res_lon = np.diff(self.lon[1:-1]).mean() * (np.pi / 180) * R_earth
        res_time = np.diff(self.time[1:-1]).mean() * day
        return res_time, res_lat, res_lon


box = Box(get_data(os.path.join(path, fn1)))

In [7]:
box[2::12].resolution

(1.0, 207.37861992560408, 208.49048745854762)

In [8]:
def smooth(box, data, sigma_t, sigma_lat, sigma_lon):
    res_t, res_lat, res_lon = box.resolution
    outp = np.zeros_like(data)
    gaussian_filter(data, [sigma_t / res_t, sigma_lat / res_lat, 0.0], output=outp)
    
    data[:] = outp
    h = data.shape[1] / 2
    for i in range(data.shape[1]):
        gaussian_filter(
            data[:,i,:],
            min(data.shape[2], (sigma_lon /res_lon) / np.cos((h - i) / (2*h) * np.pi)),
            output=outp[:,i,:])

    return outp

smooth_data = smooth(box[2::12], data[2::12], 10.0, 200, 200)

In [9]:
def plot_cube(plot_data):
    n, m = plot_data.min(), plot_data.max()
    fig = figure(x_range=(0, plot_data.shape[1]), y_range=(0, plot_data.shape[2]), plot_width=768, plot_height=384)
    clm = LinearColorMapper(palette=bokeh.palettes.viridis(255), low=n, high=m)
    img = fig.image(image=[plot_data[0,:,:]], x=0, y=0, dw=plot_data.shape[1], dh=plot_data.shape[2], color_mapper = clm)

    def update(y):  # , m):
        t = (y - 2006) # * 12 + m
        img.data_source.data['image'] = [plot_data[t]]
        push_notebook()

    show(fig, notebook_handle=True)
    interact(update, y=(2006, 2006+plot_data.shape[0] - 1))

plot_cube(smooth_data)

interactive(children=(IntSlider(value=2153, description='y', max=2300, min=2006), Output()), _dom_classes=('widget-interact',))

In [10]:
def full_sobel(box, smooth_data, weights):
    dim = len(smooth_data.shape)
    y = [w * r for r, w in zip(box.resolution, weights)]
    
    sb = np.array([
        ndimage.sobel(smooth_data, mode=['reflect', 'reflect', 'wrap'], axis=i) * y[i]
        for i in range(dim)])
    sb = np.r_[sb, np.ones_like(sb[0:1])]
    norm = np.sqrt((sb[:dim]**2).sum(axis=0))
    sb /= norm
    
    return sb

In [11]:
sb = full_sobel(box[2::12], smooth_data, weights=[100., 1./208, 1./208])

In [12]:
from hyper_canny import cp_edge_thinning, cp_double_threshold

In [19]:
dat = sb.transpose([3,2,1,0]).copy()
mask = cp_edge_thinning(dat)
edges = cp_double_threshold(data=dat, mask=mask, a=1./4000, b=2./4000)

In [28]:
fig = figure(x_range=(0, data.shape[2]), y_range=(0, data.shape[1]), plot_width=800, plot_height=500)
s_max = 1./dat[:,:,:,3].min()
s_min = 1./dat[:,:,:,3].max()
clm = LinearColorMapper(palette=bokeh.palettes.viridis(255), low=0, high=1)
#clm = LinearColorMapper(palette=bokeh.palettes.viridis(255), low=s_min, high=s_max)
img = fig.image(image=[data[0]], x=0, y=0, dw=data.shape[2], dh=data.shape[1],
                color_mapper=clm
                #palette=bokeh.palettes.grey(3)[::-1])
               )

s_l = 1.0
s_h = 2.0

def update(t, low, high):
    global s_l, s_h, m
    
    if abs(s_l - low) > 0.001 or abs(s_h - high) > 0.001:
        s_l, s_h = low, high
        m = cp_double_threshold(data=dat, mask=mask, a=1./s_h, b = 1./s_l)
        m = m.transpose([2,1,0])
        
    # img.data_source.data['image'] = [1./dat.transpose([2,1,0,3])[t,:,:,3]]
    img.data_source.data['image'] = [m[t]]
    push_notebook()

In [35]:
show(fig, notebook_handle=True)
interact(update, t=(0, dat.shape[0]-1), low=(s_min, s_max), high=(s_min, s_max))

interactive(children=(IntSlider(value=95, description='t', max=191), FloatSlider(value=773.7535302505646, description='low', max=1547.0875795117217, min=0.41948098940732254), FloatSlider(value=773.7535302505646, description='high', max=1547.0875795117217, min=0.41948098940732254), Output()), _dom_classes=('widget-interact',))

<function __main__.update>

In [34]:
plot_image(np.sqrt(m.sum(axis=0)))

In [37]:
plot_image((np.indices(m.shape)[0]*m).max(axis=0))